<a href="https://colab.research.google.com/github/from1pandas-importLove/Market_Agents_using_FinRL_python/blob/master/FinRLipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Market agents using FinRL

### Install Dependencies

In [ ]:
# # Run twice if for some reason the process is stopped

# !pip install wrds
# !pip install swig
# !pip install -q condacolab
# import condacolab
# condacolab.install()
# !apt-get update -y -qq && apt-get install -y -qq cmake libopenmpi-dev python3-dev zlib1g-dev libgl1-mesa-glx swig
# !pip install git+https://github.com/AI4Finance-Foundation/FinRL.git
# !pip install torch_geometric
# !pip install optuna
# !pip install streamlit
# !pip install quantstats
# !pip install pandas-market-calendars

###Import Libraries

In [ ]:
# import librabries and set the device
import torch
import numpy as np
import pandas as pd
from sklearn.preprocessing import MaxAbsScaler
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import GroupByScaler
from finrl.meta.env_portfolio_optimization.env_portfolio_optimization import PortfolioOptimizationEnv
from finrl.agents.portfolio_optimization.models import DRLAgent
from finrl.agents.portfolio_optimization.architectures import EIIE
import optuna

# Set device
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

print(device)


cuda:0


###Stage 1: Preprocess the data

In [ ]:
# You can use the follow implementation for the stage 1
# or use the one you implemented on stage 1

# ------------------------------------------------------------------------------------------------

# Define your custom stock list
# EX: "AAPL", "MSFT", "GOOGL", "AMZN"
# Choose the ones, that you want to work with
CUSTOM_STOCK_LIST = [
    "AAPL", "MSFT", "GOOGL", "AMZN", "FB",
    "TSLA", "BRK.B", "JNJ", "V", "WMT"
]


# Download stock data
START_DATE = '2011-01-01' # start date of the portfolio
END_DATE = '2022-12-31' # end  date of the portfolio
portfolio_raw_df = YahooDownloader(start_date=START_DATE,
                                   end_date=END_DATE,
                                   ticker_list=CUSTOM_STOCK_LIST).fetch_data()

# Group by ticker and count occurrences
portfolio_raw_df.groupby("tic").count()

# Normalize the data
# You can use GroupByScaler with a MaxAbsScaler here
portfolio_norm_df = GroupByScaler(by="tic", scaler=MaxAbsScaler).fit_transform(portfolio_raw_df)

# Select relevant columns
df_portfolio = portfolio_norm_df[["date", "tic", "close", "high", "low"]]

# Split data into training and testing sets

START_DATE_TRAIN = "2011-01-01" # you start date for the train data
END_DATE_TRAIN = "2019-12-31" # your end date for the train data
START_DATE_TEST = "2020-01-01" # your start date for the test data
END_DATE_TEST = "2022-12-31" # your end date for the test data

df_portfolio_train = df_portfolio[(df_portfolio["date"] >= START_DATE_TRAIN) & (df_portfolio["date"] < END_DATE_TRAIN)]
df_portfolio_test = df_portfolio[(df_portfolio["date"] >= START_DATE_TEST) & (df_portfolio["date"] < END_DATE_TEST)]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['FB']: YFTzMissingError('possibly delisted; no timezone found')
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['BRK.B']: YFTzMissingError('possibly delisted; no timezone found')
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Shape of DataFrame:  (24160, 8)



/usr/local/lib/python3.11/site-packages/finrl/meta/preprocessor/preprocessors.py:100: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.23665012 0.16431745 0.13584257 ... 0.04542185 0.04024656 0.0409539 ]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  X.loc[select_mask, self.columns] = self.scalers[value].transform(
/usr/local/lib/python3.11/site-packages/finrl/meta/preprocessor/preprocessors.py:100: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.   0.25 0.5  ... 0.5  0.75 1.  ]' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  X.loc[select_mask, self.columns] = self.scalers[value].transform(


In [ ]:
# Print the train and test dfs shape
TRAIN_DF_SHAPE = df_portfolio_train.shape
TEST_DF_SHAPE = df_portfolio_test.shape
print("Train df shape: ", TRAIN_DF_SHAPE)
print("Test df shape: ", TEST_DF_SHAPE)

Train df shape:  (18104, 5)
Test df shape:  (6048, 5)


### Stage 2: Train the model


In [ ]:
# Define the environment
# We will use portfolio optimization for the project
INITIAL_AMOUT = 100000 # initial amount of money in the portfolio: float
COMISSION_FEE_PTC = 0.001 # commission fee: float
TIME_WINDOW = 50# time window: int
FEATURES = ["close", "high", "low"] # ex: "close", "high"

environment_train = PortfolioOptimizationEnv(
    df_portfolio_train,
    initial_amount=INITIAL_AMOUT,
    comission_fee_pct=COMISSION_FEE_PTC,
    time_window=TIME_WINDOW,
    features=FEATURES,
    normalize_df=None # df is already normalized
)

# Set PolicyGradient parameters
# Set the learning rate for the training
model_kwargs = {
    "lr": 0.01, # put a learning rate, ex: 0.01
    "policy": EIIE, # we will use EIIE policy for this project
}

# Set EIIE's parameters
policy_kwargs = {
    "k_size": 3, # put the k_size: int
    "time_window": TIME_WINDOW, # time window defined previously
}

# Instantiate the model
model = DRLAgent(environment_train).get_model("pg", device, model_kwargs, policy_kwargs)

# Train the model
EPISODES = 10 # number of episodes to training the model: in
DRLAgent.train_model(model, episodes=EPISODES)

# Save the model
torch.save(model.train_policy.state_dict(), "policy_EIIE.pt")

  0%|          | 0/10 [00:00<?, ?it/s]

Initial portfolio value:100000
Final portfolio value: 820336.4375
Final accumulative portfolio value: 8.203364372253418
Maximum DrawDown: -0.24264830417636618
Sharpe ratio: 1.3011602735030035


findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
 10%|█         | 1/10 [00:21<03:12, 21.41s/it]

Initial portfolio value:100000
Final portfolio value: 1038208.75
Final accumulative portfolio value: 10.382087707519531
Maximum DrawDown: -0.5115777816303695
Sharpe ratio: 0.946750318976561


findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
 20%|██        | 2/10 [00:42<02:47, 20.99s/it]

...

findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
 80%|████████  | 8/10 [02:49<00:42, 21.19s/it]

Initial portfolio value:100000
Final portfolio value: 1827986.75
Final accumulative portfolio value: 18.27986717224121
Maximum DrawDown: -0.5350792878476867
Sharpe ratio: 0.9214243908088292


findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
 90%|█████████ | 9/10 [03:10<00:21, 21.07s/it]

Initial portfolio value:100000
Final portfolio value: 1828583.75
Final accumulative portfolio value: 18.285837173461914
Maximum DrawDown: -0.5350837911565954
Sharpe ratio: 0.9209394676393484


findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
100%|██████████| 10/10 [03:33<00:00, 21.34s/it]


In [ ]:
# print the final value of the portfolio
final_portfolio_value_train = environment_train._asset_memory["final"][-1]
print("The final portfolio value at train is:", final_portfolio_value_train)

The final portfolio value at train is: 1828583.8


### evaluate the model

In [ ]:
# Evaluate the model
# Create test env

INITIAL_AMOUT = 100000 # initial amount of money in the portfolio: float
COMISSION_FEE_PTC = 0.001 # commission fee: float
TIME_WINDOW = 50# time window: int
FEATURES = ["close", "high", "low"] # ex: "close", "high"

enviroment_test = PortfolioOptimizationEnv(
    df_portfolio_test,
    initial_amount=INITIAL_AMOUT,
    comission_fee_pct=COMISSION_FEE_PTC,
    time_window=TIME_WINDOW,
    features=FEATURES,
    normalize_df=None # df is already normalized
)


EIIE_results = {
    "train": environment_train._asset_memory["final"],
    "test": {},
}

# instantiate an architecture with the same arguments used in training
# and load with load_state_dict.
policy = EIIE(time_window=TIME_WINDOW, device=device)
policy.load_state_dict(torch.load("policy_EIIE.pt"))

# testing
DRLAgent.DRL_validation(model, enviroment_test, policy=policy)
EIIE_results["test"] = enviroment_test._asset_memory["final"]

Initial portfolio value:100000
Final portfolio value: 614795.125
Final accumulative portfolio value: 6.147951126098633
Maximum DrawDown: -0.6094408036581442
Sharpe ratio: 1.3693922574111272


findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.


In [ ]:
# print the final value of the portfolio
final_portfolio_value_test = enviroment_test._asset_memory["final"][-1]
print("The final portfolio value at test is:", final_portfolio_value_test)

The final portfolio value at test is: 614795.1


### Stage 4: Hyperparameter tuning with optuna


In [ ]:
EPISODES = 10# number of episodes to train the model

# Define the objective function
def objective(trial):
    lr = trial.suggest_loguniform("lr", 1e-5, 1e-2) # use trial to suggest values for lr
    k_size = trial.suggest_categorical("k_size", [3, 5, 7]) # use trial to suggest values for k_size
    time_window = trial.suggest_int("time_window", 30, 60, step=10) # use trial to suggest values for time_window

    # set up train environment
    # Training environment
    environment_train = PortfolioOptimizationEnv(
    df_portfolio_train,
    initial_amount=INITIAL_AMOUT,
    comission_fee_pct=COMISSION_FEE_PTC,
    time_window=time_window,
    features=FEATURES,
    normalize_df=None
    )

    # setup model kwargs
    model_kwargs = {
        "lr": lr,
        "policy": EIIE,
    }
    # setup policy kwargs
    policy_kwargs = {
        "k_size": k_size,
        "time_window": time_window,
    }
    # Train model using DRLAgent
    model = DRLAgent(environment_train).get_model("pg", device, model_kwargs, policy_kwargs)
    DRLAgent.train_model(model, episodes=EPISODES)

    # define test environment
    environment_test = PortfolioOptimizationEnv(
    df_portfolio_test,
    initial_amount=INITIAL_AMOUT,
    comission_fee_pct=COMISSION_FEE_PTC,
    time_window=time_window,
    features=FEATURES,
    normalize_df=None)

    # validate with test environment
    DRLAgent.DRL_validation(model, environment_test)

    # final portfolio value as metric for the optimization
    return environment_test._asset_memory["final"][-1]


# Create a study and optimize the objective function
study = optuna.create_study(direction='maximize')
N_TRIALS = 20# select the number of trails for the optimization
study.optimize(objective, n_trials=N_TRIALS)

[I 2025-04-23 04:33:21,017] A new study created in memory with name: no-name-d703026c-b772-4f32-8532-29a4d729ea53
<ipython-input-9-77ffd919438e>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform("lr", 1e-5, 1e-2) # use trial to suggest values for lr
  0%|          | 0/10 [00:00<?, ?it/s]

Initial portfolio value:100000
Final portfolio value: 684342.0
Final accumulative portfolio value: 6.843420028686523
Maximum DrawDown: -0.15010654645887667
Sharpe ratio: 1.4401996549413572


findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
 10%|█         | 1/10 [00:20<03:05, 20.61s/it]

Initial portfolio value:100000
Final portfolio value: 745700.9375
Final accumulative portfolio value: 7.457009315490723
Maximum DrawDown: -0.17684941881895588
Sharpe ratio: 1.3806679308293086


findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
 20%|██        | 2/10 [00:41<02:45, 20.68s/it]

...

findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
 90%|█████████ | 9/10 [03:08<00:21, 21.03s/it]

Initial portfolio value:100000
Final portfolio value: 1184310.0
Final accumulative portfolio value: 11.843099594116211
Maximum DrawDown: -0.3945894040336333
Sharpe ratio: 1.1066970396200726


findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
100%|██████████| 10/10 [03:28<00:00, 20.81s/it]


Initial portfolio value:100000
Final portfolio value: 253341.171875
Final accumulative portfolio value: 2.533411741256714
Maximum DrawDown: -0.3403867138341625
Sharpe ratio: 1.2869025137684167


findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
[I 2025-04-23 05:45:30,361] Trial 19 finished with value: 253341.171875 and parameters: {'lr': 0.003973102009682248, 'k_size': 5, 'time_window': 40}. Best is trial 12 with value: 414389.4375.


In [ ]:
# print the best hyperparameters
BEST_HYPERPARAMETERS = {'lr': 0.009989562388670565, 'k_size': 5, 'time_window': 50} # using the study get the best hyperparameters
print("Best hyperparameters: ", BEST_HYPERPARAMETERS)

Best hyperparameters:  {'lr': 0.009989562388670565, 'k_size': 5, 'time_window': 50}
